In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imprting Libraries

In [3]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=de33715a968bb22cff53f509fc5382f6163cc4933c7dde243ba7c137586246d3
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [4]:
from seqeval.metrics import classification_report as seq_classification_report
from seqeval.metrics import f1_score as seq_f1_score

In [5]:
def evaluate_ner_model(model, data_loader, device, idx2tag):
    model.eval()
    
    true_tags_list = []
    pred_tags_list = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            tags = batch['tags'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Get predictions
            pred_tags = model(input_ids, attention_mask)
            
            # Convert to list of tags
            for i in range(len(pred_tags)):
                length = attention_mask[i].sum().item()
                
                true_tags = [idx2tag[tag.item()] for tag in tags[i][:length]]
                pred_tags_i = [idx2tag[tag.item()] for tag in pred_tags[i][:length]]
                
                true_tags_list.append(true_tags)
                pred_tags_list.append(pred_tags_i)
    
    # Calculate metrics
    report = seq_classification_report(true_tags_list, pred_tags_list)
    f1 = seq_f1_score(true_tags_list, pred_tags_list)
    
    return report, f1

# Evaluate model


In [6]:
import os
os.listdir('/kaggle/working/')

['.virtual_documents']

In [20]:
# Step 1: Recreate the model architecture with the same parameters you used during training
model = BiLSTM_CRF(len(ner_train_dataset.word2idx), ner_train_dataset.tag2idx,
    embedding_dim=100,
    hidden_dim=256,
    num_layers=2,
    dropout=0.5)

# Step 2: Load the state dict
model.load_state_dict(torch.load('/kaggle/input/ner/pytorch/default/1/bilstm_crf_ner.pt'))

# Step 3: Send model to device
model.to(device)

/tmp/ipykernel_31/2607168004.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/ner/pytorch/default/1/bilstm_crf_ner.pt'))


BiLSTM_CRF(
  (word_embeds): Embedding(10951, 100, padding_idx=0)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (hidden2tag): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
report, f1 = evaluate_ner_model(
    model=model,
    data_loader=ner_val_loader,
    device=device,
    idx2tag=ner_train_dataset.idx2tag
)

print(f"F1 Score: {f1:.4f}")
print("Classification Report:")
print(report)

# Evaluating text classification model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import torch
from tqdm import tqdm

def evaluate_text_classifier(model, data_loader, device, class_names):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Forward pass
            logits = model(input_ids, attention_mask)
            
            # Get predictions
            _, preds = torch.max(logits, 1)
            
            # Add to lists
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    report = classification_report(all_labels, all_preds, target_names=class_names)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    
    return report, conf_matrix

# Evaluate model
class_names = text_classification_dataset['train'].features['label'].names
report, conf_matrix = evaluate_text_classifier(
    model=trained_text_classifier,
    data_loader=text_val_loader,
    device=device,
    class_names=class_names
)

print("Classification Report:")
print(report)



In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()